In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 23s 11ms/step - loss: 0.2186 - accuracy: 0.9359 - val_loss: 0.1001 - val_accuracy: 0.9687
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0972 - accuracy: 0.9704 - val_loss: 0.0985 - val_accuracy: 0.9704
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0695 - accuracy: 0.9782 - val_loss: 0.0718 - val_accuracy: 0.9779
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0526 - accuracy: 0.9830 - val_loss: 0.0701 - val_accuracy: 0.9789
Epoch 5/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0437 - accuracy: 0.9858 - val_loss: 0.0624 - val_accuracy: 0.9819


In [6]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 1204), started 0:00:30 ago. (Use '!kill 1204' to kill it.)

<IPython.core.display.Javascript object>

In [14]:
!kill 1204

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [9]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [10]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24523073434829712, Accuracy: 92.79166412353516, Test Loss: 0.12128029018640518, Test Accuracy: 96.41999816894531
Epoch 2, Loss: 0.10351218283176422, Accuracy: 96.92833709716797, Test Loss: 0.09171819686889648, Test Accuracy: 97.36000061035156
Epoch 3, Loss: 0.07234274595975876, Accuracy: 97.8183364868164, Test Loss: 0.07409212738275528, Test Accuracy: 97.68000030517578
Epoch 4, Loss: 0.05656522139906883, Accuracy: 98.23333740234375, Test Loss: 0.06700704991817474, Test Accuracy: 97.8699951171875
Epoch 5, Loss: 0.043361369520425797, Accuracy: 98.59832763671875, Test Loss: 0.0658702477812767, Test Accuracy: 97.94999694824219


In [15]:
%tensorboard --logdir logs/gradient_tape

<IPython.core.display.Javascript object>